## TensorFlow: Single Hidden Layer NNs with Interactions

In [5]:
import tensorflow as tf
import numpy as np

### Create Toy Dataset

In [6]:
np.random.seed(123)

Xtr = np.random.normal(0, 5, (2500, 3))
Xts = np.random.normal(0, 5, (2500, 3))

def f(x):
    y = (x[:, 1] + 2*x[:, 2] - x[:, 0]  # linear with no interactions
        + x[:, 0]*x[:, 1] + x[:, 0]*x[:, 1]*x[:, 2]  # linear with interactions
#          + x[:, 1]*(x[:, 2]**2)  # quadratic with interactions
        )
    return y.reshape(-1,1)

ytr = f(Xtr)
yts = f(Xts)

### Create Model

In [7]:
# hyper parameters
learning_rate = 0.01
n_input = 3
n_nodes_1 = 32
n_output = 1
batch_size = 100

# create placeholders for graph input
# the 'None' allows us to pass different batch sizes
X = tf.placeholder('float32', [None, n_input])
Y = tf.placeholder('float32', [None, n_output])


def create_weights(shape):
    # create layer weights as a variable
    # use random_normal initializer and keep the values small by setting
    # the standard devaition to 0.1 (default is 1.0)
    initializer=tf.random_normal(shape, stddev=0.1)
    W = tf.Variable(initializer)
    return W
    
def create_biases(shape):
    # create the bias parameter as varaible
    # should be rank 1 tensor with shape same as n_nodes of layer
    initializer=tf.random_normal(shape)
    b = tf.Variable(initializer)
    return b



# assign the weights we want for this model
weights = {'w1': create_weights([n_input, n_nodes_1]),  # first and only hidden layer weights
           'w_out': create_weights([n_nodes_1, n_output])}  # output layer weights

# assign biases
biases = {'b1': create_biases([n_nodes_1]),
          'b_out': create_biases([n_output])}



# Define the network forward propogation

# hidden layer
z1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])  # argument for the activation function
a1 = tf.nn.sigmoid(z1)  # the activation function


# output layer
logits = tf.add(tf.matmul(a1, weights['w_out']), biases['b_out'])  # operates on previous layer outputs
# yhat = tf.nn.softmax(logits)  # gives class probabilities


# Back propogation
# define loss function
# RMSE
loss = tf.sqrt(tf.reduce_mean(tf.square(Y-logits)))

# define optimizer
# optimize = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
optimize = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)


# create session and initialize variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


# train over 300 epochs
n_epochs = 300
for epoch in range(1, n_epochs + 1):
    # train on one batch at a time
    for i in range(0, len(Xtr), batch_size):
        sess.run(optimize, feed_dict={X: Xtr[i: i+batch_size],
                                      Y: ytr[i: i+batch_size]})
    
    # compute training loss for printing progress
    if (epoch%10 == 0) | (epoch == 1):
        loss_tr = sess.run(loss, feed_dict={X: Xtr, Y: ytr})
        print('Epoch {}, loss: {:.3f}'.format(epoch, loss_tr, 3))
        

# compute loss for test data
loss_ts = sess.run(loss, feed_dict={X: Xts, Y: yts})
print(10*'-')
print('Test loss: {:.3f}'.format(np.round(loss_ts, 3)))

Epoch 1, loss: 116.054
Epoch 10, loss: 112.855
Epoch 20, loss: 110.045
Epoch 30, loss: 108.159
Epoch 40, loss: 106.599
Epoch 50, loss: 105.201
Epoch 60, loss: 103.916
Epoch 70, loss: 102.705
Epoch 80, loss: 101.545
Epoch 90, loss: 100.458
Epoch 100, loss: 99.429
Epoch 110, loss: 98.434
Epoch 120, loss: 97.519
Epoch 130, loss: 96.657
Epoch 140, loss: 95.786
Epoch 150, loss: 94.996
Epoch 160, loss: 94.242
Epoch 170, loss: 93.524
Epoch 180, loss: 92.833
Epoch 190, loss: 92.109
Epoch 200, loss: 91.410
Epoch 210, loss: 90.715
Epoch 220, loss: 90.064
Epoch 230, loss: 89.451
Epoch 240, loss: 88.866
Epoch 250, loss: 88.304
Epoch 260, loss: 87.761
Epoch 270, loss: 87.237
Epoch 280, loss: 86.728
Epoch 290, loss: 86.235
Epoch 300, loss: 85.759
----------
Test loss: 106.336


In [8]:
abs(yts).mean()

69.96254122169644